# Cognitive Services for Computer Vision - Facial Recognition
### Talk in TDC SP 2019 Event<br>Track: Cognitive Computing<br>Date: Jun, 16 2019

**Comparison between four vendors:**
- Amazon Rekognition
- Microsoft Face API
- IBM Watson Visual Recognition
- Chooch

**After this comparison, you'll be able to:**
- Store images in a "database" for using in facial recognition
- "Train" model with faces database, generating face embeddings (faceprints)
- Run face recognition on a new image
- Draw Bounding Boxes for face matches, with info (confidence/accuracy, name)
- Draw Bounding Boxes for face unmatches, with "unknown label"


**Let's start ... Importing libraries**

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import boto3 # AWS SDK
import cognitive_face as CF # Microsoft Face API SDK
from ibm_watson import VisualRecognitionV3 as VR # IBM Visual Recognition SDK
import cv2 # Open CV
from imutils import paths # Image utils from pyimagesearch (Adrian Rosebrock)
from matplotlib import pyplot as plt

import collections
import time
import zipfile
import requests
import json
import os
import configparser
import time

## 0. Helper Functions

In [ ]:
def image2bytes(filename):
    """ 
    Convert a imagefile in bytes

    Parameters:
    filename (str): Image file to convert (with relative path to notebook folder)

    Returns:
    bytes: Array of bytes from image file

    """
        
    with open(filename, 'rb') as image_file:
        image_bytes = image_file.read()
    
    return image_bytes

In [ ]:
def read_config_file(config_file):
    """ 
    Read config file with API Keys for Cognitive Services

    Parameters:
    config_file (str): Config file to read (inside config folder of this project)

    Returns:
    config (dict) - Dictionary for Config File

    """
    
    config = configparser.RawConfigParser()
    config.read('../config/' + config_file)  
    
    return config

In [ ]:
def get_config_section(config):
    """ 
    Return config sections populated by read_config_file function

    Parameters:
    config (dict) - Config sections

    Returns:
    config_key (str) - Content of specific key in specific section at config file

    """
    
    if not hasattr(get_config_section, 'section_dict'):
        get_config_section.section_dict = dict()
        for section in config.sections():
            get_config_section.section_dict[section] = dict(config.items(section))
            
    return get_config_section.section_dict

In [ ]:
def create_zipfiles(dataset):
    
    """ 
    Create Zip files for each folder (Person) in dataset

    Parameters:
    dataset (str): Folder containing image dataset, relative to this project folder

    Returns:
    subdirs (array): List of subdirs (Person Names)

    """
    
    start_ts = time.time()

    subdirs = [o for o in os.listdir('../' + dataset) if os.path.isdir(os.path.join('../' + dataset,o))]
    
    for name in subdirs:
        zf = zipfile.ZipFile('../' + dataset + "/" + name + ".zip", "w")
        files = [x for x in os.listdir('../' + dataset + "/" + name)]
        for filename in files:
            zf.write(os.path.join('../' + dataset + "/" + name, filename), filename)
        zf.close()
        
    print(f"Zipping time: {time.time()-start_ts} seconds")
        
    return subdirs

In [ ]:
# Read Config Files
config = read_config_file('config.properties')
config_dict = get_config_section(config)

In [ ]:
# Variables for this experiment
person_group = 'persongrouptest'          # Person Group Id for Microsoft Face API
person_group_name = 'Person Group Test'   # Person Group Name for Microsoft Face API
collection_id = 'rektest01'               # Collection Id for Amazon Rekognition
classifier_name = 'faces_classifier'      # Classifier Name for IBM Watson Visual Recognition

## 1. Store images for using in a facial recognition task

Collect pictures with faces, label them with person names and finally store in database, bucket or storage, in a way it can possible to apply some embeddding / encoding algorithm to transform this face in a faceprint.

**Amazon Rekognition:**
- Store images in a S3 Bucket, label them with person name, using metadata field

In [ ]:
def configure_aws():
    """ 
    Configure access to AWS Services

    Parameters:
    none

    Returns:
    client_rekognition (boto3.client): Client for Amazon Rekognition
    client_s3 (boto3.client): Client for AWS S3

    """
    
    # Create client of boto3 (AWS SDK) for Amazon Rekognition
    client_rekognition = boto3.client(
        "rekognition", 
        aws_access_key_id=config_dict['Amazon']['access_key_id'],
        aws_secret_access_key=config_dict['Amazon']['secret_access_key'],
        region_name=config_dict['Amazon']['region_name']
    )
    
    # Create client of boto3 (AWS SDK) for AWS S3
    client_s3 = boto3.client(
        "s3", 
        aws_access_key_id=config_dict['Amazon']['access_key_id'],
        aws_secret_access_key=config_dict['Amazon']['secret_access_key'],
        region_name=config_dict['Amazon']['region_name']
    )
    
    return client_rekognition, client_s3

In [ ]:
def store_images_s3(dataset, client_s3, bucket_s3):
    """
    Read a image dataset and store in AWS S3 Bucket

    Parameters:
    dataset (str): Folder containing image dataset, relative to this project folder
    client_s3 (boto3.client): AWS S3 Client
    bucket_s3 (str): Name of bucket inside AWS S3 Service

    Returns:
    none

    """
    
    start_ts = time.time()
    
    # list folders with person names, inside dataset folder
    imagePaths = list(paths.list_images('../' + dataset))
    
    # iterate into images dataset
    for (i, imagePath) in enumerate(imagePaths):
        
        print("Storing image {}/{} in S3".format(i + 1, len(imagePaths)))
        # extract the person name from the image path
        personName = imagePath.split(os.path.sep)[-2]
        fileName = imagePath.split(os.path.sep)[-1]
        imageFile = open(imagePath,'rb')
        
        # Call AWS S3 Function to Store Image in Bucket
        ret = client_s3.put_object(Bucket=bucket_s3, 
            Key=personName+'-'+fileName, 
            Body=imageFile,
            Metadata={'FullName':personName}
        )
    
    print(f"Storing time: {time.time()-start_ts} seconds")

In [ ]:
# run code for store images - AWS
rekognition, s3 = configure_aws()
store_images_s3('dataset', s3, config_dict['Amazon']['bucket_s3'])

**Microsoft Face API:**
- Create Person Group object to store all Persons to be identified
- Create Person object into Person Group
- Detect Face in each image
- Add images into Person Group and Person, using bounding box

In [ ]:
def configure_faceapi():
    
    """ 
    Configure access to Microsoft Face API

    Parameters:
    none

    Returns:
    none

    """
    
    # Set Key to SDK
    KEY = config_dict['Microsoft']['key']
    CF.Key.set(KEY)

    # Set BASE_URL to SDK    
    BASE_URL = config_dict['Microsoft']['base_url']
    CF.BaseUrl.set(BASE_URL)    

In [ ]:
def store_images_azure(dataset, person_group, person_group_desc):
    """
    Read a image dataset and store in Azure Face API Service

    Parameters:
    dataset (str): Folder containing image dataset, relative to this project folder
    person_group (str): Code / Mnemonic for Person Group
    person_group_desc (str): Description of Person Group

    Returns:
    none

    """
    
    start_ts = time.time()

    # Create person Group
    CF.person_group.create(person_group, person_group_desc)

    # list folders with person names, inside dataset folder
    imagePaths = list(paths.list_images('../' + dataset))
    
    currentPerson = None
    
    # iterate into images dataset
    for (i, imagePath) in enumerate(imagePaths):
        print("Storing image metadata {}/{} in Azure".format(i + 1, len(imagePaths)))
        # extract the person name from the image path
        personName = imagePath.split(os.path.sep)[-2]
        
        if personName != currentPerson:
            person_id = CF.person.create(person_group, personName)
            currentPerson = personName
        
        faces = CF.face.detect(imagePath)
        if len(faces) > 0:
            top = faces[0]['faceRectangle']['top']
            left = faces[0]['faceRectangle']['left']
            width = faces[0]['faceRectangle']['width']
            height = faces[0]['faceRectangle']['height']
            
            bounding_box = str(left)+","+str(top)+","+str(width)+","+str(height)
                        
            CF.person.add_face(imagePath, person_group, person_id['personId'], target_face=bounding_box)
        
        time.sleep(6)
                
    print(f"Storing time: {time.time()-start_ts} seconds")

In [ ]:
# run code for store images - Microsoft Azure
configure_faceapi()
store_images_azure('dataset', person_group, person_group_name)

**IBM Watson Visual Recognition:**
- Create Zip file for each Person in Dataset

In [ ]:
def configure_watson():
    """ 
    Configure access to IBM Watson Visual Recognition Service

    Parameters:
    none

    Returns:
    visual_recognition (VisualRecognitionV3): Client for IBM Watson Recognition

    """
    
    # Create Visual Recognition Service
    visual_recognition = VR(
        version=config_dict['IBM']['version'],
        iam_apikey=config_dict['IBM']['iam_api_key']
    )
    return visual_recognition

In [ ]:
# run code for store images (Creating Local Zip Files) - IBM
client_watson = configure_watson()
person_names_watson = create_zipfiles('dataset_ibm')

**Chooch**

- Create Person in Catalog
- Add Images into Person

In [ ]:
def configure_chooch():
    """ 
    Configure access to Chooch

    Parameters:
    none

    Returns:
    api_key (str): API Key for Chooch

    """
    
    api_key = config_dict['Chooch']['api_key']
    
    return api_key

In [ ]:
def store_images_chooch(api_key, dataset, model_id):
    """
    Read a image dataset and store in Chooch Image Dataset

    Parameters:
    api_key (str): API Key for access Chooch
    dataset (str): Folder containing image dataset, relative to this project folder
    model_id (int): Model ID created in Chooch web interface 

    Returns:
    none

    """
    
    start_ts = time.time()
    
    # list folders with person names, inside dataset folder
    imagePaths = list(paths.list_images('../' + dataset))
    
    currentPerson = None
    
    # iterate into images dataset
    for (i, imagePath) in enumerate(imagePaths):
        print("Storing image metadata {}/{} in Chooch".format(i + 1, len(imagePaths)))
        # extract the person name from the image path
        personName = imagePath.split(os.path.sep)[-2]
        
        if personName != currentPerson:
            # Create Person
            url_create_person = 'https://api.chooch.ai/predict/face?person_name='+personName+'&model_id='+str(model_id)+'&apikey='+api_key+'&command=create_person'
            response = requests.post(url_create_person)
            response_create_person = json.loads(response.content)
                        
            if (response_create_person['status_description'] == 'success'):
                person_id = response_create_person['person_id']
                currentPerson = personName

        # Add Image to Person
        url_add_image = 'https://api.chooch.ai/predict/face?person_id_filter='+str(person_id)+'&apikey='+api_key+'&command=insert_person_image'
        image_file = {'image': open(imagePath, 'rb')}
        response = requests.post(url_add_image, files=image_file)
        response_add_image = json.loads(response.content)
                                        
    print(f"Storing time: {time.time()-start_ts} seconds")

In [ ]:
# run code for store images - Chooch
api_key = configure_chooch()
store_images_chooch(api_key, 'dataset', config_dict['Chooch']['model_id'])

## 2. "Train" model with faces database, generating face embeddings (faceprints)

Read a collection of images from storage and generate embeddding / encoding algorithm to transform this face in a faceprint.

**Amazon Rekognition:**
- Create a collection in rekognition, which generates embedding about each face detected in each image from bucket
- Store Face ID generated by embedding task as key with label (person name) from image in a key-value database (DynamoDB)

In [ ]:
def train_rekognition(client_rekognition, client_s3, bucket_s3, collection):
    """
    Train a image dataset, generating faces index

    Parameters:
    client_rekognition (boto3.client): Client for Amazon Rekognition
    client_s3 (boto3.client): Client for AWS S3
    bucket_s3 (str): Bucket name in AWS S3
    collection (str): Id for Collection for Face Indexes

    Returns:
    none

    """
    
    start_ts = time.time()
    
    # create collection on Rekognition
    print("Creating collection in Rekognition")
    client_rekognition.create_collection(CollectionId=collection)
    
    # get list of images from S3
    print("Get list of images from S3")
    imageList = client_s3.list_objects_v2(
        Bucket=bucket_s3,
    )
    
    # for each file from S3
    for (i, file) in enumerate(imageList['Contents']):
        print("Embedding image {}/{} from S3".format(i + 1, len(file) + 1))
        personName = file['Key'].split('-')[-2]
        # generate embeddings using IndexFaces method
        face_embedding = client_rekognition.index_faces(
                            CollectionId=collection,
                            Image={
                                'S3Object': {
                                    'Bucket': bucket_s3,
                                    'Name': file['Key']
                                }
                            },
                            ExternalImageId=personName,
                            DetectionAttributes=['DEFAULT'],
                            MaxFaces=1
                        )
        
    print(f"Training time: {time.time()-start_ts} seconds")

In [ ]:
# run code for embedding - AWS
train_rekognition(rekognition, s3, config_dict['Amazon']['bucket_s3'], collection_id)

**Microsoft Face API:**
- Train model in Person Group

In [ ]:
def train_faceapi(person_group):
    """
    Train a Person Group

    Parameters:
    person_group (str): Name of Person Group

    Returns:
    none

    """
    
    start_ts = time.time()
    
    CF.person_group.train(person_group)
    
    i = 1
    status = 'Pending'
    
    while (status != 'succeeded'):
        print("Training - Step {}".format(i))
        response = CF.person_group.get_status(person_group)
        status = response['status']
        i =+ 1
        
    print(f"Training time: {time.time()-start_ts} seconds")

In [ ]:
# run code for training - Microsoft Face API
train_faceapi(person_group)

**IBM Watson Visual Recognition:**
- Create a Classifier with images, naming each Classifier with Person Name

In [ ]:
def create_watson_classifier(client, person_names, dataset, classifier):
    """
    Create and train a image classifier, using dataset

    Parameters:
    client (VisualRecognitionV3): Client for IBM Watson Visual Recognition
    names (str): Array of Person Names, obtained in dataset subfolders
    dataset (str): Folder containing image dataset, relative to this project folder
    classifier (str): Name of Classifier

    Returns:
    model (json): Information about new classifier. https://cloud.ibm.com/apidocs/visual-recognition#create-a-classifier

    """
    
    start_ts = time.time()
    
    positiveClasses = {}
    
    names = [x for x in person_names if x != 'negative']
    
    # Open ZIP for positive classes (Person Names)
    for person in names:
        print('Creating Dict for Person Name: {}'.format(person))
        positiveClasses[person] = open('../' + dataset + '/' + person + '.zip', 'rb')
        
    # Open ZIP for negative class
    negativeClasses = open('../' + dataset + '/negative.zip', 'rb')
    
    # Create 
    model = client.create_classifier(
        name=classifier,
        positive_examples=positiveClasses,
        negative_examples=negativeClasses)
        
    print(f"Training time: {time.time()-start_ts} seconds")
        
    return model

In [ ]:
# run code for creating classifier - Watson Visual Recognition
watson_classifier = create_watson_classifier(client_watson, person_names_watson, 'dataset_ibm', classifier_name)

In [ ]:
watson_classifier_id = watson_classifier.result['classifier_id']
print(watson_classifier_id)
print(watson_classifier.result['status'])

## 3. Run face recognition on a new image

With a new captured image, execute facial recognition, search new image in embeddings database

In [ ]:
new_image_known_face = '../test/T0004.jpg'
new_image_unknown_face = '../test/u0001.jpg'

**Amazon Rekognition:**
- Search for faces in a collection similar to face detected in the new image

In [ ]:
def face_recognition_aws(client_rekognition, image, collection, threshold):
    """
    Execute Face Recognition in Collection trained in Amazon Rekognition

    Parameters:
    client_rekognition (boto3.client): Client for Amazon Rekognition
    image (str): Image file to recognize face (with relative path to notebook folder)
    collection (str): Name of the trained collection
    threshold (int): Threshold for Face Recognition

    Returns:
    personname (str): Person Name
    bounding_box (dict): Bounding box with face detected in image. Format: Top, Left, Width, Height
    similarity (float): Similarity level in percentage (accuracy)

    """
    
    start_ts = time.time()
    
    # Search collection using image
    known_faces = client_rekognition.search_faces_by_image(
        CollectionId=collection,
        Image={
            'Bytes': image2bytes(image)
        },
        FaceMatchThreshold=threshold
    )
    
    img1 = cv2.imread(image)
    imgHeight, imgWidth, _ = img1.shape
    
    box = known_faces['SearchedFaceBoundingBox']
    
    left = int(imgWidth * box['Left'])
    top = int(imgHeight * box['Top'])
    width = int(imgWidth * box['Width'])
    height = int(imgHeight * box['Height'])
    
    bounding_box = {'Top': top, 'Left': left, 'Width': width, 'Height': height}
    
    if len(known_faces['FaceMatches']) > 0:
        personname = known_faces['FaceMatches'][0]['Face']['ExternalImageId']
        similarity = known_faces['FaceMatches'][0]['Similarity']
    else:   
        personname = 'Unknown'
        similarity = 0.0
        
    print(f"Recognition time: {time.time()-start_ts} seconds")
    
    return personname, bounding_box, similarity

In [ ]:
# run facial recognition (Known Face) - AWS
personname_aws, bounding_box_aws, similarity_aws = face_recognition_aws(rekognition, new_image_known_face, collection_id, 90)

In [ ]:
# run facial recognition (Unknown Face) - AWS
personname_aws_u, bounding_box_aws_u, similarity_aws_u = face_recognition_aws(rekognition, new_image_unknown_face, collection_id, 90)

**Microsoft Face API:**
- Run face detect and identify using data stored and trained in Person Group

In [ ]:
def face_recognition_microsoft(person_group, image, threshold):
    """
    Execute Face Recognition in Person Group defined in Azure Cognitive Face API

    Parameters:
    person_group (str): Name of Person Group
    image (str): Image file to recognize face (with relative path to notebook folder)
    threshold (int): Threshold for Face Recognition

    Returns:
    personname (str): Person Name
    bounding_box (dict): Bounding box with face detected in image. Format: Top, Left, Width, Height
    confidence (float): Confidence level in percentage (accuracy)

    """
    
    start_ts = time.time()

    # Run Face Detection with new image
    faces = CF.face.detect(image)
    face_ids = [f['faceId'] for f in faces]
    
    if len(faces) > 0:
        top = faces[0]['faceRectangle']['top']
        left = faces[0]['faceRectangle']['left']
        width = faces[0]['faceRectangle']['width']
        height = faces[0]['faceRectangle']['height']

        bounding_box = {'Top': top, 'Left': left, 'Width': width, 'Height': height}
    
        face_matches = CF.face.identify(face_ids, person_group)
        
        if len(face_matches[0]['candidates']) > 0:
            person = CF.person.get(person_group, face_matches[0]['candidates'][0]['personId'])
            personname = person['name']
            confidence = face_matches[0]['candidates'][0]['confidence']
        else:
            personname = 'Unknown'
            confidence = 0.0
            
    print(f"Recognition time: {time.time()-start_ts} seconds")
    
    return personname, bounding_box, confidence

In [ ]:
# run facial recognition (Known Face) - Microsoft Face API
personname_microsoft, bounding_box_microsoft, confidence_microsoft = face_recognition_microsoft(person_group, new_image_known_face, 90)

In [ ]:
# run facial recognition (Unknown Face) - Microsoft Face API
personname_microsoft_u, bounding_box_microsoft_u, confidence_microsoft_u = face_recognition_microsoft(person_group, new_image_unknown_face, 90)

**IBM Watson Visual Recognition:**
- Run face detection on new photo
- Classify new image, using classifier

In [ ]:
def face_detection_watson(client, image):
    """
    Detect Faces in a image, using IBM Watson Visual Recognition

    Parameters:
    client (VisualRecognitionV3): Client for IBM Watson Visual Recognition
    image (str): Image file to recognize face (with relative path to notebook folder)

    Returns:
    bounding_box (dict): Bounding box with face detected in image. Format: Top, Left, Width, Height

    """
    
    start_ts = time.time()
    
    # Run Face Detection with new image
    
    with open(image, 'rb') as images_file:
        faces = client.detect_faces(images_file).get_result()
    
    if len(faces['images'][0]['faces']) > 0:
        top = faces['images'][0]['faces'][0]['face_location']['top']
        left = faces['images'][0]['faces'][0]['face_location']['left']
        width = faces['images'][0]['faces'][0]['face_location']['width']
        height = faces['images'][0]['faces'][0]['face_location']['height']

        bounding_box = {'Top': top, 'Left': left, 'Width': width, 'Height': height}
        
    print(f"Face Detection time: {time.time()-start_ts} seconds")

    return bounding_box

In [ ]:
def classify_image_watson(client, classifier_id, image, threshold):
    """
    Run Image Classifier from IBM Watson Visual Recognition, using image

    Parameters:
    client (VisualRecognitionV3): Client for IBM Watson Visual Recognition
    classifier_id (str): Classifier Id returned from training step
    image (str): Image file to recognize face (with relative path to notebook folder)
    threshold (int): Threshold for Image Classification

    Returns:
    person (str): Person Name
    bounding_box (dict): Bounding box with face detected in image. Format: Top, Left, Width, Height
    score (float): Score level in percentage (accuracy)


    """
    
    start_ts = time.time()

    # Run Classifier against new image
    threshold = threshold / 100
    
    with open(image, 'rb') as images_file:
        faces = client.classify(
            images_file,
            threshold='0.9',
            classifier_ids=[classifier_id]).get_result()
        
    if len(faces['images'][0]['classifiers'][0]['classes']) > 0:
        person = faces['images'][0]['classifiers'][0]['classes'][0]['class']
        score = faces['images'][0]['classifiers'][0]['classes'][0]['score']
    else:
        person = 'Unknown'
        score = 0.0
        
    print(f"Recognition time: {time.time()-start_ts} seconds")
    
    return person, score

In [ ]:
# run facial detection (Known Face) - IBM Watson
bounding_box_watson = face_detection_watson(client_watson, new_image_known_face)

In [ ]:
# run visual recognition (Known Face) - IBM Watson
personname_watson, score_watson = classify_image_watson(client_watson, watson_classifier_id, new_image_known_face, 90)

In [ ]:
# run facial detection (Unknown Face) - IBM Watson
bounding_box_watson_u = face_detection_watson(client_watson, new_image_unknown_face)

In [ ]:
# run visual recognition (Unknown Face) - IBM Watson
personname_watson_u, score_watson_u = classify_image_watson(client_watson, watson_classifier_id, new_image_unknown_face, 90)

**Chooch:**

- Run face recognition using data stored and trained in Perception

In [ ]:
def face_recognition_chooch(api_key, image, model_id):
    """
    Execute Face Recognition in Perception defined in Chooch Web App
    
    Parameters:
    api_key (str): API Key for access Chooch
    image (str): Image file to recognize face (with relative path to notebook folder)
    model_id (int): Model ID created in Chooch web interface 

    Returns:
    personname (str): Person Name
    bounding_box (dict): Bounding box with face detected in image. Format: Top, Left, Width, Height
    Similarity (float): Similarity level in percentage (accuracy)

    """
    
    start_ts = time.time()
    
    # Run Face Recognition with new image
    url_face_recognition = 'https://api.chooch.ai/predict/face?model_id='+str(model_id)+'&apikey='+api_key
    image_file = {'image': open(image, 'rb')}
    response = requests.post(url_face_recognition, files=image_file)
    response_face_recognition = json.loads(response.content)
    
    X1, X2, Y1, Y2 = response_face_recognition['faces'][0]['coordinates'].split(',')
    bounding_box = {'Top': int(Y1), 'Left': int(X1), 'Width': int(X2)-int(X1), 'Height': int(Y2)-int(Y1)}

    if response_face_recognition['face_recog_hit']:
        personname = response_face_recognition['faces'][0]['person_name']
        similarity = response_face_recognition['faces'][0]['similarity']
    else:
        personname = 'Unknown'
        similarity = 0.0
        
    print(f"Recognition time: {time.time()-start_ts} seconds")
    
    return personname, bounding_box, similarity

In [ ]:
# run facial recognition (Known Face) - Chooch
personname_chooch, bounding_box_chooch, similarity_chooch = face_recognition_chooch(api_key, new_image_known_face, config_dict['Chooch']['model_id'])

In [ ]:
# run facial recognition (Unknown Face) - Chooch
personname_chooch_u, bounding_box_chooch_u, similarity_chooch_u = face_recognition_chooch(api_key, new_image_known_face, config_dict['Chooch']['model_id'])

## 4. Draw Bounding Boxes for face matches, with info (confidence/accuracy, name)

Read data from facial recognition, check face matches and draw a bounding box with name and confidence/accuracy in the new image

**Amazon / Microsoft / IBM / Chooch:**
- Use a helper function (OpenCV) to draw a bounding box, passing facial recognition data, image and box info (Person Name and Accuracy)

In [ ]:
def draw_bounding_boxes(box, personname, accuracy, image):
    """
    Draw Bounding Box in image, with Person Name or Unknown and Accuracy Level
    
    Parameters:
    bounding_box (dict): Bounding box with face detected in image. Format: Top, Left, Width, Height
    personname (str): Person Name
    accuracy (float): Accuracy level in percentage
    image (str): Image file to draw bounding box (with relative path to notebook folder)

    Returns:
    none
    
    """

    # display the image to our screen
    img1 = cv2.imread(image)

    left = box['Left']
    top = box['Top']
    width = box['Width']
    height = box['Height']
    
    label = personname + '/' + str(accuracy)

    cv2.rectangle(img1, (left, top), (left + width, top + height), (255, 0, 0), 5)
    y = top - 15 if top - 15 > 15 else top + 15
        
    cv2.putText(img1, label, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 4.00, (255, 0, 0), 5)

    plt.imshow(img1)
    plt.title('Image')
    plt.show()

In [ ]:
# run draw bounding boxes (face matches) - AWS
draw_bounding_boxes(bounding_box_aws, personname_aws, similarity_aws, new_image_known_face)

In [ ]:
# run draw bounding boxes (face matches) - Microsoft
draw_bounding_boxes(bounding_box_microsoft, personname_microsoft, confidence_microsoft, new_image_known_face)

In [ ]:
# run draw bounding boxes (face matches) - Watson
draw_bounding_boxes(bounding_box_watson, personname_watson, score_watson, new_image_known_face)

In [ ]:
# run draw bounding boxes (face matches) - Chooch
draw_bounding_boxes(bounding_box_chooch, personname_chooch, similarity_chooch, new_image_known_face)

## 5. Draw Bounding Boxes for face unmatches, with unkwown tag

Read data from facial recognition, check face unmatches and draw a bounding box with unknown tag in the new image

**Amazon Rekognition:**
- Use a helper function (OpenCV) to draw a bounding box, passing face recognition data, image and box info (Unknown)

In [ ]:
# run draw bounding boxes (face unmatches) - AWS
draw_bounding_boxes(bounding_box_aws_u, personname_aws_u, 0.0, new_image_unknown_face)

In [ ]:
# run draw bounding boxes (face unmatches) - Microsoft
draw_bounding_boxes(bounding_box_microsoft_u, personname_microsoft_u, 0.0, new_image_unknown_face)

In [ ]:
# run draw bounding boxes (face unmatches) - Watson
draw_bounding_boxes(bounding_box_watson_u, personname_watson_u, 0.0, new_image_unknown_face)

In [ ]:
# run draw bounding boxes (face unmatches) - Chooch
draw_bounding_boxes(bounding_box_chooch_u, personname_chooch_u, 0.0, new_image_unknown_face)

**Useful links that helped me build this notebook**

- https://aws.amazon.com/blogs/machine-learning/build-your-own-face-recognition-service-using-amazon-rekognition/
- https://www.pyimagesearch.com/2019/03/25/building-a-raspberry-pi-security-camera-with-opencv/
- https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html
- https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html
- https://docs.aws.amazon.com/rekognition/latest/dg/what-is.html
- https://docs.microsoft.com/en-us/azure/cognitive-services/Face/Tutorials/FaceAPIinPythonTutorial
- https://docs.microsoft.com/en-us/python/api/overview/azure/cognitiveservices/faceapi?view=azure-python
- https://github.com/microsoft/Cognitive-Face-Python/
- https://clemenssiebler.com/face-recognition-with-azure-cognitive-services-face-api/
- https://cloud.ibm.com/apidocs/visual-recognition?code=python
- https://chooch.ai/api/